In [221]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [222]:
# Load from dataset for testing
def load_dataset(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    laser_data = []
    odometry_data = []
    
    for line in lines:
        if line.startswith('FLASER'):
            line = line.strip().split()
            num_laser_values = int(line[1])  # Extracting number of laser values
            values = line[2:]  # Extracting laser and odometry values
            max_range = 10
            scan_data = np.clip([float(value) for value in values[:num_laser_values]], 0, max_range + 1)
            scan = Scan(angle_min=-np.pi/2, angle_max=np.pi/2, range_max=max_range, angle_increment=(1 * np.pi) / float(num_laser_values), ranges=scan_data)
            laser_data.append(scan)
            #laser_data.append(scan_data)
            odometry_data.append([float(value) for value in values[num_laser_values:num_laser_values+3]])
    
    return np.array(laser_data), np.array(odometry_data)
scans, poses = load_dataset('orebro.gfs.log')

In [223]:
from slam.fast_slam import FastSLAM
slam = FastSLAM()
%lprun -f FastSLAM.update slam.update(np.zeros(2), poses[0], scans[0])
# Loop time 1.6

250000
(1.2792799999999998, 0.08173649999999999) (0, 0) (11, 10) (11, 10)
2.2630908991235827 -2.8019160056086756 2 -2 (0, 0) (11, 10)


AssertionError: Y is outside of search range

In [ ]:
%lprun -f OccupancyGrid.update slam.particles[0].map.update(poses[0], scans[0])

In [ ]:
particle = slam.particles[0]
%lprun -f LikelihoodField.__init__  LikelihoodField(particle.map)

In [ ]:
test_map = slam.particles[0].map.get_map() > 0.5
np.argwhere((np.isreal(test_map)[5:10,5:10]))

In [ ]:
test_map = slam.particles[5].map.get_map() == 0.5
fig, ax = plt.subplots()
im = ax.imshow(test_map, cmap='jet')
ax = plt.gca()
ax.set_xlim([0, 40])
ax.set_ylim([0, 60])
fig.colorbar(im)

In [ ]:
np.unique(slam.particles[0].map.test_map)

In [ ]:
grid = OccupancyGrid()
for i in range(100):
    grid.update(poses[i] + np.array([5, 5, 0]), scans[i])
    plt.imshow(grid.get_map(), cmap='gray')
    ax = plt.gca()
    ax.set_xlim([0, 300])
    ax.set_ylim([0, 100])
    plt.savefig('out/%03d.png' % i)